In [6]:
import json

def count_jsonl_entries(file_path):
    """
    Counts the number of entries in a .jsonl (JSON Lines) file.

    :param file_path: Path to the .jsonl file
    :return: Number of entries in the file
    """
    count = 0
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                if line.strip():  # Ignore empty lines
                    count += 1
    except FileNotFoundError:
        print(f"Error: The file at {file_path} does not exist.")
        return 0
    except Exception as e:
        print(f"An error occurred: {e}")
        return 0
    
    return count


# Example usage
file_path = "../results/multiturn_task_x_specialties_x_demographic_x_answerstyle.jsonl"
num_entries = count_jsonl_entries(file_path)
print(f"The .jsonl file contains {num_entries} entries.")

The .jsonl file contains 20395 entries.
